# Homework for next week: Introduction to Data Science for Sports

You will exercise your knowledge of probabilities and write down logic with python.

<br />

**Question 1** (20 points) Here are the driver [standings](https://www.formula1.com/en/drivers.html) for the year 2021. What a year it was! Mercedes' Lewis Hamilton *lost* to Red Bull Racing's Max Verstappen!. Here are the last 3 F1 races for the season, that happened while we were learning data science! 
- Italy GP: Date: September 10
- Russia GP: Date: September 24
- Turkey GP: Date: October 10

Suppose we are just starting the semester, right before the Italy Grand Prix (this coming weekend) and the Russia Grand Prix the weekend after, as you can see [schedule](https://www.formula1.com/en/racing/2021.html). 

The 2021 driver standings are already [tabulated](https://www.formula1.com/en/drivers.html) and Max beat Lewis! But let's ***assume*** these standings ***before the Italy GP***. 

Given these standings (please do not use team standings given on the same Web site, use ***driver standings***), what is the Probability Distribution for each F1 driver to win the Italy Grand Prix? 

What is the Probability Distribution for each F1 driver to win ***both*** the Italy and the Russia Grand Prix? 

What is the probability for Red Bull Racing to win ***both*** races? 

What is the probability for Red Bull Racing to win ***at least*** one race? 

What is the probability for Red Bull Racing to win ***all three*** races? 

Note that Red Bull Racing, and each other racing team, has ***two*** drivers per race. Use individual driver standings (not team standings), and use the same *final* [standings](https://www.formula1.com/en/results.html/2020/drivers.html) for both races.

**Question 2** (30 points) If Red Bull Racing wins the first race, what is the probability that Red Bull Racing wins the next one? If Red Bull Racing wins at least one of these two races, what is the probability Red Bull Racing wins both races? How about Mercedes, McLaren, and Ferrari?

**Question 3** (50 points) Red Bull Racing wins at least one of these two races on a ***rainy day***. What is the probability Red Bull Racing wins ***both*** races, assuming races can be held on either rainy, sunny, cloudy, snowy or foggy days? Also assume that rain, sun, clouds, snow, and fog are the *only possible weather conditions* on race tracks, and that they're *equiprobable*.

You need to provide *proof* for your answers. `I think it's one in a million because Red Bull Racing ***sucks*** and I like Mercedes ***a lot more***` is not a good answer. Leverage the counting framework in this workbook!

Define function that return probability distribution of each and every item

In [47]:
class ProbDist(dict):
    """A Probability Distribution; an {outcome: probability} mapping."""
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        # Make probabilities sum to 1.0; assert no negative probabilities
        total = sum(self.values())
        for outcome in self:
            self[outcome] = self[outcome] / total
            assert self[outcome] >= 0

**IGP, RGP, and weather** stored the probabilty distribution of italia GP, Russian GP, and wheather

**teamPlayer** stored the player belongs to which team

In [48]:
IGP = ProbDist(
    MV = 287.5,
    LH = 275.5,
    VB = 185,
    LN = 149,
    SP = 150,
    CS = 122.5,
    CL = 128,
    DR = 105,
    PG = 74,
    FA = 58,
    EO = 46,
    SV = 36,
    LS = 26,
    YT = 20,
    GR = 16,
    NL = 7,
    KR = 6,
    AG = 1,
    MS = 0,
    NM = 0
)

RGP = IGP
teamPlayer = {
    'MV': 'Redbull',
    'SP': 'Redbull',
    'LH': 'Merc',
    'VB': 'Merc',
    'LN': 'McL',
    'CS': 'Fer',
    'CL': 'Fer',
    'DR': 'McL',
    'PG': 'AlpT',
    'FA': 'Alp',
    'EO': 'Alp',
    'SV': 'AM',
    'LS': 'AM',
    'YT': 'AlpT',
    'GR': 'Will',
    'NL': 'Will',
    'KR': 'AlpR',
    'AG': 'AlpR',
    'MS': 'Haas',
    'NM': 'Haas'
}

weather = ProbDist(
    rain = 1, 
    sun = 1,
    cloudy = 1,
    snow = 1,
    fog = 1
)

Defined function **teamJoin** which return team dictionary along with team performance 

In [49]:
team = {}
def teamJoin():
    for i,v in teamPlayer.items():
        if(v not in team):
            team[v] = IGP[i]
        else:
            team[v]+= IGP[i]
teamJoin()
team = ProbDist(team)

In [50]:
def joint(A, B, sep=''):
    """The joint distribution of two independent probability distributions. 
    Result is all entries of the form {a+sep+b: P(a)*P(b)}"""
    return ProbDist({a + sep + b: A[a] * B[b]
                    for a in A
                    for b in B})

varible stored all the possible combination of the players and teams

In [51]:
P2R = joint(IGP, RGP , ' ')
T2R = joint(team, team, ' ')
T3R = joint(team, joint(team, team, ' ') , ' ')

Defined function **first_race_winner, second_race_winner, and third_race_winner** return the outcome who win the race at given position i.e. player or team name in the given position

Defined function **removeData** is used to delete the data which is not needed

In [52]:
def first_race_winner(outcome, key): return outcome.split(' ')[0] == key
def second_race_winner(outcome, key): return outcome.split(' ')[1] == key
def third_race_winner(outcome, key): return outcome.split(' ')[2] == key
def removeData(outcome, key): return outcome.count(key) == 0

In [53]:
def p_and(event_collection, space, x): 
    """The probability of an event collection intersection, given a sample space of non-equiprobable outcomes. 
    event: a collection of predicates that are true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs.
    x: key on which event has to be applied"""
    l = list(map(lambda e: {o for o in space if e(o, x)}, event_collection))
    return sum([space[o] for o in l[0].intersection(*l)])

def p(event, space, x): 
    """The probability of an event, given a sample space of equiprobable outcomes. 
    event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs.
    x: key on which event has to be applied"""
    # branch on the type of the first argument
    if is_predicate(event):
        event = such_that(event, space, x)
    if isinstance(space, ProbDist):
        return sum([space[o] for o in event])
    else:
        return Fraction(len(event & space), len(space))

is_predicate = callable

def such_that(predicate, space, x):
    """The outcomes in the sample space for which the predicate is true.
    If space is a set, return a subset {outcome,...} with outcomes where predicate(element, key) is true;
    if space is a ProbDist, return a ProbDist {outcome: frequency,...} with outcomes where predicate(element, key) is true."""
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o, x)})
    else:
        return {o for o in space if predicate(o, x)}

Given these standings (please do not use team standings given on the same Web site, use driver standings), what is the Probability Distribution for each F1 driver to win the Italy Grand Prix?

In [54]:
IGP

{'MV': 0.16986706056129985,
 'LH': 0.16277695716395865,
 'VB': 0.10930576070901034,
 'LN': 0.0880354505169867,
 'SP': 0.08862629246676514,
 'CS': 0.0723781388478582,
 'CL': 0.07562776957163958,
 'DR': 0.0620384047267356,
 'PG': 0.04372230428360414,
 'FA': 0.03426883308714919,
 'EO': 0.027178729689807977,
 'SV': 0.021270310192023634,
 'LS': 0.01536189069423929,
 'YT': 0.011816838995568686,
 'GR': 0.009453471196454948,
 'NL': 0.00413589364844904,
 'KR': 0.0035450516986706058,
 'AG': 0.0005908419497784342,
 'MS': 0.0,
 'NM': 0.0}

What is the Probability Distribution for each F1 driver to win both the Italy and the Russia Grand Prix?

# Solution

Mapping as per the teamPlayer keys and **p_and** is passing two event first and second winner, **P2R** is player probability of two races, and **x** is the player name.

**p_add** will return probability value of player winning both the races along with values

**W2R** stored dictionary with player name and probability winning both the races.

In [55]:
W2R = dict(map(lambda x: (
    x, p_and(
        {
            first_race_winner, 
            second_race_winner
        }, P2R, x)),
               teamPlayer.keys()))

W2R

{'MV': 0.028854818263736343,
 'SP': 0.007854619716404599,
 'LH': 0.02649633778355726,
 'VB': 0.011947749324175442,
 'LN': 0.0077502405477288235,
 'CS': 0.005238594983079846,
 'CL': 0.005719559530381019,
 'DR': 0.0038487636610382544,
 'PG': 0.0019116398918680708,
 'FA': 0.0011743529211548924,
 'EO': 0.0007386833475516505,
 'SV': 0.00045242609566490495,
 'LS': 0.00023598768570175598,
 'YT': 0.00013963768384719292,
 'GR': 8.936811766220343e-05,
 'NL': 1.710561627128113e-05,
 'KR': 1.2567391546247361e-05,
 'AG': 3.4909420961798214e-07,
 'MS': 0.0,
 'NM': 0.0}

What is the probability for Red Bull Racing to win both races?

# Solution

**p_add** will return probability value of Redbull winning both the races 

In [56]:
p_and(
    {
        first_race_winner, 
        second_race_winner
    }, T2R, 'Redbull')

0.06681881355969181

What is the probability for Red Bull Racing to win all three races?

# Solution

**p_add** will return probability value of Redbull winning all the three races 

In [57]:
p_and(
    {
        first_race_winner, 
        second_race_winner, 
        third_race_winner
    }, T3R, 'Redbull')

0.0172722191624019

What is the probability for Red Bull Racing to win at least one race?

# Solution

**teamWithNoWin** stored the dictionary with possibility of all the teams winning none races

**atleast1W3R** stored the dictionary with possibility of teams who has atleast one win in all the three race

We are mapping on team keys list by taking each team not winning any race and subtracting by 1

**logic**

n = 3

A = team winning atleast 1 race 

B = Other team winning the race

**$$P(A) = 1 - P(B) ^ n$$**

In [58]:
teamWithNoWin = dict(map(lambda x: 
                         (x, 
                          p(removeData,team, x)
                         ), team.keys()))

atleast1W3R = dict(map(lambda x: 
                       (x, 
                        (1 - (teamWithNoWin[x] ** 3)
                        )), team.keys()))
atleast1W3R

{'Redbull': 0.5922958375675216,
 'Merc': 0.6143031505046624,
 'McL': 0.3860350673319488,
 'Fer': 0.38154265874681315,
 'AlpT': 0.1575349563723928,
 'Alp': 0.3211326980409567,
 'AM': 0.10592000564841664,
 'Will': 0.04021659158342705,
 'AlpR': 0.012356434843542874,
 'Haas': 0.0}

If Red Bull Racing wins the first race, what is the probability that Red Bull Racing wins the next one? How about Mercedes, McLaren, and Ferrari?

# Solution 

**P2W** stored dictionary the probability of a team winning both runs

**PT2TW** stored dictionary the probability of team wins the second race if first is race is won

We are running mapping on findTeam list by taking each team winning both the races and dividing the value by winning one race

**Logic**

A = Team win the second race

B = Team win first race

$$ P(A|B) = \frac{P(AnB)}{P(B)}$$

In [59]:
findTeam = ['Redbull','Merc','McL','Fer']
P2W = dict(map(lambda x: 
               (x, 
                (p_and(
                    {
                        first_race_winner, 
                        second_race_winner
                    }, T2R, x)
                )), findTeam))

PT2TW = dict(map(lambda x: (x,  P2W[x] / team[x]), findTeam))
PT2TW

{'Redbull': 0.258493353028065,
 'Merc': 0.272082717872969,
 'McL': 0.1500738552437223,
 'Fer': 0.1480059084194978}

If Red Bull Racing wins at least one of these two races, what is the probability Red Bull Racing wins both races? How about Mercedes, McLaren, and Ferrari?

# Solution

**atLeastOne** stored dictionary possibility of each team winning at least one race

**BothRace** stored dictionary possibility of each team winning both the race

We are running mapping on findTeam list by taking each team intersection of 2 race wins and at least one win and dividing the value by at least one win

**Logic**

A = Both races win by a team

B = At least one race won by a team

$$ P(A|B) = \frac{P(AnB)}{P(B)}$$

In [60]:
atLeastOne = dict(map(lambda x: 
                      (x, 
                       1 - (teamWithNoWin[x] ** 2))
                      , findTeam))

BothRace = dict(map(lambda x : (x, P2W[x] / atLeastOne[x]), findTeam))
BothRace

{'Redbull': 0.1484308736217132,
 'Merc': 0.15746281415626603,
 'McL': 0.08112424145640364,
 'Fer': 0.07991705216142923}

Red Bull Racing wins at least one of these two races on a rainy day. What is the probability Red Bull Racing wins both races, assuming races can be held on either rainy, sunny, cloudy, snowy or foggy days? 

# Solution

Define function **filteringData** return filter data where at least one key is **Redbull rain**

Define function **filterRBW2Race** return data where Redbull won both races

**P2TWW** stored joint probability of team and weather condition for two race

The probability is found out by first filtering out the data set as per Redbull win at least one of the races on a rainy day then this data is pass and we further filter that data to find Redbull win both the race that returns probability value of Redbull winning both at least on one rainy date.

**Logic**

A = Both races win by Redbull team in either weather condition

B = At least one race won by Redbull team in rain weather condition

$$ P(A|B) = \frac{P(AnB)}{P(B)}$$

In [61]:
def filteringData(outcome,x): 
    return outcome.count(x) > 0
def filterRBW2Race(outcome,x): 
    return outcome.count(x) == 2

In [65]:
P2TWW = joint(joint(team,weather,' '), joint(team,weather,' '), ' ')
p(filterRBW2Race, such_that(filteringData, P2TWW, 'Redbull rain'), 'Redbull')


0.23881728582259273